## GENERAL IMPORTS

In [ ]:
%env SM_FRAMEWORK=tf.keras
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import warnings

def function_that_warns():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    function_that_warns()  # this will not show warnings

import os
import numpy as np
import sys
import tensorflow as tf
from tensorflow import keras
import glob
import cv2
from patchify import patchify, unpatchify
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score, precision_recall_fscore_support
from PIL import Image

In [ ]:
# Replace with your path
save_dir = 'YOUR_PATH'

## FINE TUNING FOR ROY

The trained model will be chosen accordingly. 
Please modify the path. 

In [ ]:
## Choose a model with the right path
model = tf.keras.models.load_model('YOUR_SAVED_MODELS_PATH/model_1_cus.h5')

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load the pre-trained UNet model
pretrained_model = model

# Replace the final layer with a new layer that predicts building locations
x = pretrained_model.layers[-2].output
x = Conv2D(1, (1, 1), activation='sigmoid')(x)
fine_tuned_model = Model(pretrained_model.input, x)

# Freeze some of the pre-trained layers
for layer in fine_tuned_model.layers[:-4]:
    layer.trainable = False

# Compile the model with a suitable loss function and optimizer
fine_tuned_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])


In [ ]:
# Split the data from path_in into training and validation sets
# Creating Images and masks path and deleting images whose masks didnt exist
count = 0
train_ids_paths=[]
train_masks_path=[]
base = 'YOUR_PATH_TO_YOUR_TRAINING_DATA' # The data is in the form of images and labels, where labels are further divided into the relevant classes.

total_maps= os.listdir(base)
t = sorted(glob.glob(os.path.join(base, 'images', '*.png')))
m = sorted(glob.glob(os.path.join(base, 'labels','0', '*.png')))

print(len(t), len(m))

total_imgs=[]
total_masks=[]

for item in t:
    t1=item.split('/')
    total_imgs.append(t1[-1])

for item in m:
    m1=item.split('/')
    total_masks.append(m1[-1]) 

list_difference = []
for item in total_imgs:
    if item not in total_masks:
        list_difference.append(item)

for item in list_difference:
    toRemove= os.path.join(base, 'images', item)
    t.remove(toRemove)

if len(t) == len(m):
    train_ids_paths +=t
    train_masks_path += m
else:
    print( 'number of images',len(t), i) 
    print( 'number of masks',len(m), i)
print(len(train_ids_paths), len(train_masks_path))


In [ ]:
#Reading Images and masks and creating tensors respectively
Masks=[]
Images=[]  
count = 0 
with tf.device('/gpu:0'):     
  for img_path,mask_path in tqdm(zip(train_ids_paths, train_masks_path),total=len(train_ids_paths),position=0, leave=False):
    img = Image.open(img_path)
    Images.append(np.array(img))
    mask= cv2.imread(mask_path,2)
    mask= np.array(mask)
    mask = np.expand_dims(mask,-1)
    mask[mask>0]=1
    Masks.append(mask)
    count = count +1

In [ ]:
#Converting Images and masks into numpy array
Images=np.array(Images)
Masks=np.array(Masks)
print(Images.shape, Masks.shape)

#Shuffling images and masks to get rid of overfitting
from sklearn.utils import shuffle
Images, Masks=shuffle(Images, Masks, random_state=42)

#Display 10 images and respective maps
count = 0
for i in range(len(Images)):
  if count < 5:
      fig = plt.figure(figsize=(10, 7))
      rows,columns=1,2
      fig.add_subplot(rows, columns, 1)
      toShow=(1)*4  
      # showing image
      plt.imshow(Images[i])
      plt.axis('off')
      plt.title("Image")

      #Adds a subplot at the 2nd position
      fig.add_subplot(rows, columns, 2)
      plt.imshow(Masks[i][:,:,0], cmap='gray')
      plt.axis('off')
      plt.title("Mask")
  count +=1

print (count)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

kf = KFold(n_splits = 3)
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

fold_var = 1
count=0

for train_index, val_index in kf.split(Images,Masks):
    training_data = Images[train_index]
    validation_data = Images[val_index]

    training_mask = Masks[train_index]
    validation_mask = Masks [val_index]
    
    model = fine_tuned_model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model_name= 'model_ROY'+str(fold_var)+'.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+"FINE_TUNED_model_"+str(fold_var)+".h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks = [
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
              tf.keras.callbacks.TensorBoard(log_dir="logs"),checkpoint
        ]
    
    history = model.fit(x=training_data, y = training_mask, batch_size=64, epochs=10, verbose = 0, validation_data=(validation_data, validation_mask),callbacks=callbacks)	
    model.load_weights(save_dir+"FINE_TUNED_model_"+str(fold_var)+".h5")
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    results = model.evaluate(validation_data,validation_mask)
    results = dict(zip(model.metrics_names,results))
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    tf.keras.backend.clear_session()
    fold_var += 1

## APPLY TO PREDICT IMAGES

In [ ]:
from tensorflow.keras.models import load_model

gt_folder = 'YOUR_PATH_TO_YOUR_TEST_DATA'

def calculate_metrics(pred, gt):
    # Binarize the masks
    gt[gt > 0] = 1
    pred[pred > 0] = 1

    # Compute precision, recall, and F1 score
    precision, recall, f1_score, _ = precision_recall_fscore_support(gt.ravel(), pred.ravel(), average='binary')
    
    # Compute IoU
    IoU = jaccard_score(gt.ravel(), pred.ravel(), average='binary')
    
    return precision, recall, f1_score, IoU

# define 9 thresholds:
threshold_values = [0.01, 0.012, 0.014, 0.016, 0.018, 0.020, 0.022, 0.024, 0.026]

# Load the model
model = load_model(save_dir + "FINE_TUNED_model2.h5")
